Neural Network used to predict the delivery time of food.

In [0]:
pip install tensorflow==2.0.0

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd /content/drive/My Drive/Fast Food Hackathon

/content/drive/My Drive/Fast Food Hackathon


In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import MaxAbsScaler
from tensorflow.keras.regularizers import l2

In [0]:
df = pd.read_csv('training.csv', index_col=0)
X_test = pd.read_csv('test.csv')


In [0]:
df = df.drop(['Location'], axis = True)
X_test = X_test.drop(['Location'], axis = True)

In [0]:
print(df.head())
print(df.info())
print(X_test.head())
print(X_test.info())

   Average_Cost  Minimum_Order  Rating  ...  Cuisine6  Cuisine7  Cuisine8
0         200.0           50.0     3.5  ...        69        69        69
1         100.0           50.0     3.5  ...        69        69        69
2         150.0           50.0     3.6  ...        69        69        69
3         250.0           99.0     3.7  ...        69        69        69
4         200.0           99.0     3.2  ...        69        69        69

[5 rows x 14 columns]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11094 entries, 0 to 11093
Data columns (total 14 columns):
Average_Cost     11094 non-null float64
Minimum_Order    11094 non-null float64
Rating           11094 non-null float64
Votes            11094 non-null float64
Reviews          11094 non-null float64
Delivery_Time    11094 non-null int64
Cuisine1         11094 non-null int64
Cuisine2         11094 non-null int64
Cuisine3         11094 non-null int64
Cuisine4         11094 non-null int64
Cuisine5         11094 non-null in

In [0]:
y = df.loc[:, 'Delivery_Time']
X = df.drop(['Delivery_Time'], axis = 1)
print(X.head())
print(y.head())

   Average_Cost  Minimum_Order  Rating  ...  Cuisine6  Cuisine7  Cuisine8
0         200.0           50.0     3.5  ...        69        69        69
1         100.0           50.0     3.5  ...        69        69        69
2         150.0           50.0     3.6  ...        69        69        69
3         250.0           99.0     3.7  ...        69        69        69
4         200.0           99.0     3.2  ...        69        69        69

[5 rows x 13 columns]
0    30
1    30
2    65
3    30
4    65
Name: Delivery_Time, dtype: int64


In [0]:
#encode the y values
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
y = encoder.fit_transform(y)
print(np.unique(y))
#hotencoding using keras
y = np_utils.to_categorical(y)


[0 1 2 3 4 5 6]


In [0]:
sc = MaxAbsScaler()
X = sc.fit_transform(X)
X_test = sc.fit_transform(X_test)


In [0]:
print(pd.DataFrame(X).head())
print(pd.DataFrame(X_test).head())

         0      1         2         3   ...        9         10        11        12
0  0.097561  0.100  0.714286  0.001325  ...  1.000000  0.683168  0.683168  0.811765
1  0.048780  0.100  0.714286  0.001215  ...  0.683168  0.683168  0.683168  0.811765
2  0.073171  0.100  0.734694  0.010934  ...  0.683168  0.683168  0.683168  0.811765
3  0.121951  0.198  0.755102  0.019439  ...  0.683168  0.683168  0.683168  0.811765
4  0.097561  0.198  0.653061  0.057544  ...  0.683168  0.683168  0.683168  0.811765

[5 rows x 13 columns]
         0    1         2         3   ...        9         10        11    12
0  0.291667  0.1  0.875000  0.046217  ...  0.683168  0.741935  0.741935  0.75
1  0.083333  0.1  0.000000  0.000000  ...  0.683168  0.741935  0.741935  0.75
2  0.083333  0.1  0.750000  0.004609  ...  0.683168  0.741935  0.741935  0.75
3  0.166667  0.1  0.750000  0.008450  ...  0.871287  0.741935  0.741935  0.75
4  0.125000  0.1  0.604167  0.004865  ...  0.683168  0.741935  0.741935  0.75

[5 r

In [0]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.097561,0.100,0.714286,0.001325,0.000615,0.306931,0.821782,0.198020,0.831683,1.000000,0.683168,0.683168,0.811765
1,0.048780,0.100,0.714286,0.001215,0.000615,0.415842,0.287129,0.683168,0.683168,0.683168,0.683168,0.683168,0.811765
2,0.073171,0.100,0.734694,0.010934,0.004613,0.465347,0.910891,0.306931,0.683168,0.683168,0.683168,0.683168,0.811765
3,0.121951,0.198,0.755102,0.019439,0.014606,0.673267,0.722772,0.257426,0.683168,0.683168,0.683168,0.683168,0.811765
4,0.097561,0.198,0.653061,0.057544,0.036132,0.217822,0.138614,0.683168,0.683168,0.683168,0.683168,0.683168,0.811765
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11089,0.121951,0.100,0.857143,0.036006,0.029059,0.970297,0.257426,0.277228,0.663366,0.683168,0.683168,0.683168,0.811765
11090,0.048780,0.100,0.734694,0.003976,0.002460,0.306931,0.683168,0.683168,0.683168,0.683168,0.683168,0.683168,0.811765
11091,0.048780,0.100,0.714286,0.004970,0.002768,0.871287,0.257426,0.722772,0.683168,0.683168,0.683168,0.683168,0.811765
11092,0.048780,0.100,0.632653,0.002651,0.001384,0.722772,0.683168,0.683168,0.683168,0.683168,0.683168,0.683168,0.811765


In [0]:
print(X.shape, y.shape, X_test.shape)

(11094, 13) (11094, 7) (2774, 13)


In [0]:
classes = np.unique([np.argmax(x) for x in y])
classes

array([0, 1, 2, 3, 4, 5, 6])

In [0]:
np.unique([np.argmax(x) for x in y], return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6]),
 array([   4,   20, 7406, 2665,  923,   14,   62]))

In [0]:
from sklearn.utils.class_weight import compute_class_weight
c_weights = compute_class_weight('balanced', classes, [np.argmax(x) for x in y])
c_dict = dict()
for i, val in enumerate(c_weights):
  c_dict[i] = val
c_dict, c_weights

({0: 396.2142857142857,
  1: 79.24285714285715,
  2: 0.21399637359669765,
  3: 0.5946931117662825,
  4: 1.7170716607336325,
  5: 113.20408163265306,
  6: 25.56221198156682},
 array([3.96214286e+02, 7.92428571e+01, 2.13996374e-01, 5.94693112e-01,
        1.71707166e+00, 1.13204082e+02, 2.55622120e+01]))

In [0]:
model = Sequential()

model.add(Dense(16, activation='relu', input_shape=(X.shape[1], )))
model.add(Dropout(0.2, noise_shape=None, seed=None))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2, noise_shape=None, seed=None ))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2, noise_shape=None, seed=None))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2, noise_shape=None, seed=None ))

model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 16)                224       
_________________________________________________________________
dropout_17 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 32)                544       
_________________________________________________________________
dropout_18 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 64)                2112      
_________________________________________________________________
dropout_19 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 128)              

In [0]:
# estimator = KerasClassifier(build_fn=baseline_model, epochs = 100, batch_size = 5, verbose = 0)
# kfold = KFold(n_splits = 5, shuffle = True)
# results = cross_val_score(estimator, X, new_y, cv = kfold)
# print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
checkpointer = ModelCheckpoint('mymodel.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')
# earlystopper = EarlyStopping(monitor='val_accuracy', patience=25)
history = model.fit(X, batch_size=16, y=y, verbose=1, validation_split=0.2, 
          shuffle=True, epochs=1000, callbacks=[checkpointer], class_weight=c_weights)

Train on 8875 samples, validate on 2219 samples
Epoch 1/1000
8800/8875 [============================>.] - ETA: 0s - loss: 0.9044 - accuracy: 0.6598
Epoch 00001: val_accuracy improved from -inf to 0.67733, saving model to mymodel.h5
8875/8875 [==============================] - 2s 196us/sample - loss: 0.9022 - accuracy: 0.6607 - val_loss: 0.8095 - val_accuracy: 0.6773
Epoch 2/1000
8752/8875 [============================>.] - ETA: 0s - loss: 0.8265 - accuracy: 0.6657
Epoch 00002: val_accuracy improved from 0.67733 to 0.68229, saving model to mymodel.h5
8875/8875 [==============================] - 1s 146us/sample - loss: 0.8269 - accuracy: 0.6650 - val_loss: 0.8062 - val_accuracy: 0.6823
Epoch 3/1000
8576/8875 [===========================>..] - ETA: 0s - loss: 0.8128 - accuracy: 0.6734
Epoch 00003: val_accuracy improved from 0.68229 to 0.69130, saving model to mymodel.h5
8875/8875 [==============================] - 1s 143us/sample - loss: 0.8116 - accuracy: 0.6744 - val_loss: 0.7860 - val_

KeyboardInterrupt: ignored

In [0]:
model = tf.keras.models.load_model('mymodel.h5')

In [0]:
predictions = model.predict(X_test)

In [0]:
y_preds = [np.argmax(x) for x in predictions]
np.unique(y_preds, return_counts=True)

(array([2, 3, 6]), array([2133,  639,    2]))

In [0]:
y_preds = encoder.inverse_transform(y_preds)
np.unique(y_preds, return_counts=True)

(array([ 30,  45, 120]), array([2133,  639,    2]))

In [0]:
result_preds = pd.DataFrame(y_preds, columns=['Delivery_Time'], index=None)
result_preds['Delivery_Time'] = result_preds['Delivery_Time'].astype(str) + ' minutes'
result_preds.to_excel('nn.xlsx')